In [2]:
from Configs.dbConfigs import PROFILE_TABLE_COLUMNS, NO_UPDATE_COLS
from Entity.profile import Profile
from Main.login import Login
import pandas as pd
import json
from Utils.gsutils import GoogleSheet
from Utils.utils import get_concatinated_df, get_df_items_for_search_results, get_uncommon_rows, remove_duplicates_from_df

In [3]:
api = Login()
df = pd.DataFrame(columns=PROFILE_TABLE_COLUMNS)
profile_id = "ACoAABQ11fIBQLGQbB1V1XPBZJsRwfK5r1U2Rzw"

In [4]:
results = api.search_people(
                            keyword_title=["Recruiter"],
                            regions=["103644278"],
                            network_depths=["S", "O"],
                            keyword_company="Google",
                            limit=20,
                            )


In [5]:
print(len(results))

19


In [6]:
dataFrameData = []
for peopleJSON in results:
    profile = Profile()
    
    peopleProfile = api.get_profile(peopleJSON["urn_id"])
    profile.parseProfileJSON(peopleProfile)
    profile.parseSearchPeopleJSON(peopleJSON)
    dataFrameData.append(get_df_items_for_search_results(profile))

In [7]:
df_people_from_search = pd.DataFrame(dataFrameData, columns=PROFILE_TABLE_COLUMNS)

In [8]:
df_people_from_search

,public_id,profile_urn_id,profile_firstName,profile_lastName,profile_network_distance,profile_latest_company,profile_latest_job_title,category,connection_req_withdrawn_status,connection_req_withdrawn_date,record_added_to_sheet
0,nolan-apfel-a434a5116,ACoAABznoZMBUQgOvmeqxTdSK-lPFVnLxw-rHsw,Nolan,Apfel,DISTANCE_2,Google,Technical Recruiter,HR,False,None,28-05-2023
1,emily-e-thomas,ACoAABReE8EBHzLkiZsIzPlwVEh_bq6Wn2Q7R2g,Emily,Thomas,DISTANCE_2,Google,Senior Technical Recruiter,HR,False,None,28-05-2023
2,chris-clemens-a4a73b158,ACoAACXXFxoBXcS_boczzcPm4PG4963ElIbVn7U,Chris,Clemens,DISTANCE_2,Google,Technical Recruiter - Security Engineering,HR,False,None,28-05-2023
3,karesaespinosa,ACoAAA22HFEBQnvjyAv-OKbYfX_pl3A4V3oOGBM,Karesa,Espinosa,DISTANCE_2,Google,Technical Recruiter,HR,False,None,28-05-2023
4,jasmyneigbodipe,ACoAAAatMx4BrGyU3mSXtAUlobEohmA911Wrxo4,Jasmyne,I.,DISTANCE_2,Google,Senior Technical Recruiter,HR,False,None,28-05-2023
5,jessica-quinn-81538412,ACoAAAKL65EBP8ZXQu7-j4piPPrjHNhwGl2bL8A,Jessica,Quinn,DISTANCE_2,Google,Recruiter,HR,False,None,28-05-2023
6,joanna-perlman,ACoAAA58XukBQjTiYP0uUTzqP8MiFye-Q7Y9wi4,Joanna,Perlman,DISTANCE_3,Google,Technical Recruiter,HR,False,None,28-05-2023
7,juan-alberto-montoya-63458278,ACoAABB1VQgB-cbnQgAZ4DaL90HLHec5dY5ZHv4,Juan Alberto,Montoya,DISTANCE_2,Google,Recruiter,HR,False,None,28-05-2023
8,jessica-calabrese-8789a47,ACoAAAFkZdwB7Wads0Ydamv3yJr_KqICYIL-6ak,Jessica,Calabrese,DISTANCE_2,Google,Recruiter,HR,False,None,28-05-2023
9,macarenadza,ACoAAAL4DIoBe3KtKM5cydLkqCcdk80VjRna3-A,Macarena,Diaz,DISTANCE_2,Google,Recruiter @ Google via Nelson,HR,False,None,28-05-2023


In [9]:
gsheet = GoogleSheet()
client = gsheet.authenticate()
df_people_from_gsheet = gsheet.get_spreadsheet_data(client)

In [16]:
new_rows_added_df = get_uncommon_rows(df_people_from_gsheet,df_people_from_search)
concatinated_df = get_concatinated_df(df_people_from_gsheet, new_rows_added_df)
no_duplicates_df = remove_duplicates_from_df(concatinated_df, unq_identifier_col='profile_urn_id')
pd.set_option('display.max_columns', None)
gsheet.update_spreadsheet(client, no_duplicates_df)

Updated data in sheet 'profile_test' of spreadsheet 'linkedin_automation_test'.


In [11]:
df_people_from_gsheet

,public_id,profile_urn_id,profile_firstName,profile_lastName,profile_network_distance,profile_latest_company,profile_latest_job_title,category,connection_req_withdrawn_status,connection_req_withdrawn_date,record_added_to_sheet
0,nolan-apfel-a434a5116,ACoAABznoZMBUQgOvmeqxTdSK-lPFVnLxw-rHsw,Nolan,Apfel,DISTANCE_2,Google,Technical Recruiter,HR,FALSE,,28-05-2023
1,emily-e-thomas,ACoAABReE8EBHzLkiZsIzPlwVEh_bq6Wn2Q7R2g,Emily,Thomas,DISTANCE_2,Google,Senior Technical Recruiter,HR,FALSE,,28-05-2023
2,chris-clemens-a4a73b158,ACoAACXXFxoBXcS_boczzcPm4PG4963ElIbVn7U,Chris,Clemens,DISTANCE_2,Google,Technical Recruiter - Security Engineering,HR,FALSE,,28-05-2023
3,karesaespinosa,ACoAAA22HFEBQnvjyAv-OKbYfX_pl3A4V3oOGBM,Karesa,Espinosa,DISTANCE_2,Google,Technical Recruiter,HR,FALSE,,28-05-2023
4,jasmyneigbodipe,ACoAAAatMx4BrGyU3mSXtAUlobEohmA911Wrxo4,Jasmyne,I.,DISTANCE_2,Google,Senior Technical Recruiter,HR,FALSE,,28-05-2023
5,jessica-quinn-81538412,ACoAAAKL65EBP8ZXQu7-j4piPPrjHNhwGl2bL8A,Jessica,Quinn,DISTANCE_2,Google,Recruiter,HR,FALSE,,26-05-2023
6,joanna-perlman,ACoAAA58XukBQjTiYP0uUTzqP8MiFye-Q7Y9wi4,Joanna,Perlman,DISTANCE_3,Google,Technical Recruiter,HR,FALSE,,26-05-2024
7,juan-alberto-montoya-63458278,ACoAABB1VQgB-cbnQgAZ4DaL90HLHec5dY5ZHv4,Juan Alberto,Montoya,DISTANCE_2,Google,Recruiter,HR,FALSE,,26-05-2025
8,jessica-calabrese-8789a47,ACoAAAFkZdwB7Wads0Ydamv3yJr_KqICYIL-6ak,Jessica,Calabrese,DISTANCE_2,Google,Recruiter,HR,FALSE,,26-05-2026
9,macarenadza,ACoAAAL4DIoBe3KtKM5cydLkqCcdk80VjRna3-A,Macarena,Diaz,DISTANCE_2,Google,Recruiter @ Google via Nelson,HR,FALSE,,26-05-2027


In [12]:
no_duplicates_df

,public_id,profile_urn_id,profile_firstName,profile_lastName,profile_network_distance,profile_latest_company,profile_latest_job_title,category,connection_req_withdrawn_status,connection_req_withdrawn_date,record_added_to_sheet
0,nolan-apfel-a434a5116,ACoAABznoZMBUQgOvmeqxTdSK-lPFVnLxw-rHsw,Nolan,Apfel,DISTANCE_2,Google,Technical Recruiter,HR,FALSE,,28-05-2023
1,emily-e-thomas,ACoAABReE8EBHzLkiZsIzPlwVEh_bq6Wn2Q7R2g,Emily,Thomas,DISTANCE_2,Google,Senior Technical Recruiter,HR,FALSE,,28-05-2023
2,chris-clemens-a4a73b158,ACoAACXXFxoBXcS_boczzcPm4PG4963ElIbVn7U,Chris,Clemens,DISTANCE_2,Google,Technical Recruiter - Security Engineering,HR,FALSE,,28-05-2023
3,karesaespinosa,ACoAAA22HFEBQnvjyAv-OKbYfX_pl3A4V3oOGBM,Karesa,Espinosa,DISTANCE_2,Google,Technical Recruiter,HR,FALSE,,28-05-2023
4,jasmyneigbodipe,ACoAAAatMx4BrGyU3mSXtAUlobEohmA911Wrxo4,Jasmyne,I.,DISTANCE_2,Google,Senior Technical Recruiter,HR,FALSE,,28-05-2023
5,jessica-quinn-81538412,ACoAAAKL65EBP8ZXQu7-j4piPPrjHNhwGl2bL8A,Jessica,Quinn,DISTANCE_2,Google,Recruiter,HR,FALSE,,26-05-2023
6,joanna-perlman,ACoAAA58XukBQjTiYP0uUTzqP8MiFye-Q7Y9wi4,Joanna,Perlman,DISTANCE_3,Google,Technical Recruiter,HR,FALSE,,26-05-2024
7,juan-alberto-montoya-63458278,ACoAABB1VQgB-cbnQgAZ4DaL90HLHec5dY5ZHv4,Juan Alberto,Montoya,DISTANCE_2,Google,Recruiter,HR,FALSE,,26-05-2025
8,jessica-calabrese-8789a47,ACoAAAFkZdwB7Wads0Ydamv3yJr_KqICYIL-6ak,Jessica,Calabrese,DISTANCE_2,Google,Recruiter,HR,FALSE,,26-05-2026
9,macarenadza,ACoAAAL4DIoBe3KtKM5cydLkqCcdk80VjRna3-A,Macarena,Diaz,DISTANCE_2,Google,Recruiter @ Google via Nelson,HR,FALSE,,26-05-2027


In [13]:
concatinated_df

,public_id,profile_urn_id,profile_firstName,profile_lastName,profile_network_distance,profile_latest_company,profile_latest_job_title,category,connection_req_withdrawn_status,connection_req_withdrawn_date,record_added_to_sheet
0,nolan-apfel-a434a5116,ACoAABznoZMBUQgOvmeqxTdSK-lPFVnLxw-rHsw,Nolan,Apfel,DISTANCE_2,Google,Technical Recruiter,HR,FALSE,,28-05-2023
1,emily-e-thomas,ACoAABReE8EBHzLkiZsIzPlwVEh_bq6Wn2Q7R2g,Emily,Thomas,DISTANCE_2,Google,Senior Technical Recruiter,HR,FALSE,,28-05-2023
2,chris-clemens-a4a73b158,ACoAACXXFxoBXcS_boczzcPm4PG4963ElIbVn7U,Chris,Clemens,DISTANCE_2,Google,Technical Recruiter - Security Engineering,HR,FALSE,,28-05-2023
3,karesaespinosa,ACoAAA22HFEBQnvjyAv-OKbYfX_pl3A4V3oOGBM,Karesa,Espinosa,DISTANCE_2,Google,Technical Recruiter,HR,FALSE,,28-05-2023
4,jasmyneigbodipe,ACoAAAatMx4BrGyU3mSXtAUlobEohmA911Wrxo4,Jasmyne,I.,DISTANCE_2,Google,Senior Technical Recruiter,HR,FALSE,,28-05-2023
5,jessica-quinn-81538412,ACoAAAKL65EBP8ZXQu7-j4piPPrjHNhwGl2bL8A,Jessica,Quinn,DISTANCE_2,Google,Recruiter,HR,FALSE,,26-05-2023
6,joanna-perlman,ACoAAA58XukBQjTiYP0uUTzqP8MiFye-Q7Y9wi4,Joanna,Perlman,DISTANCE_3,Google,Technical Recruiter,HR,FALSE,,26-05-2024
7,juan-alberto-montoya-63458278,ACoAABB1VQgB-cbnQgAZ4DaL90HLHec5dY5ZHv4,Juan Alberto,Montoya,DISTANCE_2,Google,Recruiter,HR,FALSE,,26-05-2025
8,jessica-calabrese-8789a47,ACoAAAFkZdwB7Wads0Ydamv3yJr_KqICYIL-6ak,Jessica,Calabrese,DISTANCE_2,Google,Recruiter,HR,FALSE,,26-05-2026
9,macarenadza,ACoAAAL4DIoBe3KtKM5cydLkqCcdk80VjRna3-A,Macarena,Diaz,DISTANCE_2,Google,Recruiter @ Google via Nelson,HR,FALSE,,26-05-2027


In [14]:
new_rows_added_df

,public_id,profile_urn_id,profile_firstName,profile_lastName,profile_network_distance,profile_latest_company,profile_latest_job_title,category,connection_req_withdrawn_status,connection_req_withdrawn_date,record_added_to_sheet


In [15]:
df_people_from_search[~df_people_from_search['profile_urn_id'].isin(df_people_from_gsheet['profile_urn_id'])]

,public_id,profile_urn_id,profile_firstName,profile_lastName,profile_network_distance,profile_latest_company,profile_latest_job_title,category,connection_req_withdrawn_status,connection_req_withdrawn_date,record_added_to_sheet
10,greg-geiger-53062751,ACoAAAry7DQBRD5U_bIjfuVTndtT_MfcfpNMoOg,Greg,Geiger,DISTANCE_2,Google,Senior Technical Recruiter,HR,False,None,28-05-2023
11,sofia-grach-23723b131,ACoAACA_posBXPv9QQFXGy_vfqpNAG98I_4awFE,Sofia,Grach,DISTANCE_3,Google,IT Recruiter,HR,False,None,28-05-2023
12,treywashington,ACoAACd41ScBMtWTe3-UL4sV7FL5NvthN2P4aUs,Trey,W.,DISTANCE_3,Google,Early Career Recruiter,HR,False,None,28-05-2023
13,courtney-davidson-06471ab8,ACoAABkF6MwB3XnGDTS9aNBHtLfzZ5RVwvH7NHk,courtney,davidson,DISTANCE_3,Google,Recruiter,HR,False,None,28-05-2023
14,matthew-lavoie-71a9095b,ACoAAAzFXMYBgIR0U3UX8ugzOGIjhLbEvkTF0bc,Matthew,Lavoie,DISTANCE_2,Google,Recruiter,HR,False,None,28-05-2023
15,vanessa-bacorn-7108b9114,ACoAABycG7wBrgHbCTvhlqwIodeQ5T5JEzR5PKE,VANESSA,BACORN,DISTANCE_3,Google,Recruiter,HR,False,None,28-05-2023
16,lauren-beck-1131a6a3,ACoAABXrTx8BkcKmV6EDc4HuqMokImVcENkquTg,Lauren,Beck,DISTANCE_3,Google,Technical Recruiter,HR,False,None,28-05-2023
17,stephanie-kim-206,ACoAABR8ASUBBR9YiVcqEqMFc-hS0RNBm3dTfVs,Steph,Kim,DISTANCE_3,Google,Recruiter,HR,False,None,28-05-2023
18,ninamazurova,ACoAABCsN_UBuXwIXhbgvvcaqUqelex3C-gTtzQ,Nina,Kisel,DISTANCE_2,Google,Recruiter - Software Engineering,HR,False,None,28-05-2023
